# Aprendizado de Máquina - Projeto 1
### Guilherme Pereira Corrêa 198397
### Bruno Moreira...

## Parte 1 - Métodos de clusterização

## Coleta e tratamento dos dados

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
import nltk
from nltk.corpus import twitter_samples, stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem.porter import *
import string
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import gensim.downloader
import random
from scipy.spatial import distance
from scipy import stats
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from time import perf_counter
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from collections import Counter


### Tarefa 2D padrão:

Vamos primeiro visualizar os dados da tarefa 2D padrão e entender como os mesmos estão distribuídos.

In [ ]:
def scatter_data_2D(data):
    plt.scatter(data[:,0],data[:,1])
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.show()
def scatter_data_3D(data):
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.set_xlabel('Feature 1')
    ax.set_ylabel('Feature 2')
    ax.set_zlabel('Feature 3')
    ax.scatter3D(data[:,0],data[:,1],data[:,2])

data = pd.read_csv('cluster.dat', delimiter=' ')
np_data = data.to_numpy()
print(np_data)
scatter_data_2D(np_data)

Percebe-se, então, que a feature 1 deste conjunto de dados tem uma escala muito maior que a feature 2 e, portanto, pode afetar os resultados por ter um peso maior indesejado nos cálculos de distância que serão realizados. Escalar estes dados pode resolver isso. Além disso, os dados estão distruidos crescentemente e, portanto, devem ser embaralhados na separação entre train set e test set para evitar bias.

In [ ]:
#normalizado com StandardScaler()
data_scaled = StandardScaler().fit_transform(np_data)
#normalizado com normalize
data_normalized = normalize(np_data,axis=0)
scatter_data_2D(data_scaled)

Agora vamos definir uma função para separar o dataset em training set e test set.

In [ ]:
def split_dataset(data,train_proportion,test_proportion):
    set_size = data.shape[0]
    train_size = int(set_size*train_proportion)
    train_set = data[:train_size]
    test_set = data[train_size:]
    return train_set,test_set

In [ ]:
#Dados sem pré processamento
np.random.shuffle(np_data)
train_set,test_set = split_dataset(np_data,0.9,0.1)

#Dados normalizados com StandardScaler
np.random.shuffle(data_scaled)
train_set_scaled,test_set_scaled = split_dataset(data_scaled,0.9,0.1)

#Dados normalizados com normalize
np.random.shuffle(data_normalized)
train_set_normalized,test_set_normalized = split_dataset(data_normalized,0.9,0.1)

### 2º conjunto de dados: tweets.
Foram escolhidos tweets para serem estudados como eles se comportam ao aplicar um algoritmo de clusterização. Porém, por se tratarem de textos, a forma como os mesmos são transformados em vetores de números (embedding) é crucial para o resultado. Neste projeto, esse processo foi dividido em três passos, onde existem diversas maneiras diferentes de se fazer cada um:

1 - Coleta e pré processamento dos tweets.

2 - Word embedding

3 - Document embedding

**1 - Coleta e pré processamento dos tweets**

O download dos tweets foi feito utilizando a biblioteca nltk, contendo 10000 tweets e uma anotação sobre o sentimento de cada um. Essa anotação é feita ao separar os mesmos em dois conjuntos, um para os tweets positivos e o outro para os tweets negativos, ou seja, como essa anotação não será utilizada neste trabalho, então esses dois conjuntos foram simplesmente concatenados um ao outro e então essa lista foi embaralhada a fim de evitar bias no momento de separar entre train set e test set.

O pré processamento de cada tweet foi feito ao remover stopwords, pontuação, links e hashtags, além de tokenização. A princípio não foi feito stemming pois planejava-se usar embedder pronto e o mesmo considera as palavras completas e não suas raizes. Porém, por conta da possibilidade de utilizar outro embedder estar em aberta, foram implementados duas funções de pré processamento, uma com stemming, e uma sem. 

In [ ]:
def process_tweet(tweet):
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):
            # remove punctuation
             tweets_clean.append(word)
            #stem_word = stemmer.stem(word)  # stemming word
            #tweets_clean.append(stem_word)
    return tweets_clean

def process_tweet_stem(tweet):
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):
            # remove punctuation
             #tweets_clean.append(word)
             stem_word = stemmer.stem(word)  # stemming word
             tweets_clean.append(stem_word)
    return tweets_clean

Download do tweets e embaralhamento dos mesmos.

In [ ]:
nltk.download('twitter_samples')
nltk.download('stopwords')
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')
all_tweets = all_positive_tweets + all_negative_tweets
#Shuffle para não ter bias na hora de separar entre train_set e test_set
np.random.shuffle(all_tweets)

**2 - Word embedding**

Utilizando a biblioteca gensim, foi feito o download do modelo GloVe de 200 dimensões pré treinado no Twitter, essa escolha foi feita pois esse modelo é um dos que apresentam melhores resultados atualmente e, principalmente, porque ele foi treinado no Twitter, e o contexto de treino é muito importante neste tipo de tarefa.

In [ ]:
#Embedding GloVe que será utilizado
glove_vectors = gensim.downloader.load('glove-twitter-200')

**3 - Document embedding**

Há diversas maneiras de representar um tweet numericamente, mas muitas são custosas e, por conta disso, utilizamos uma ténica chamada Bag-of-words (BOW), onde a representação de um documento é dada pela soma das representações de cada palavra que o compõe.

In [ ]:
def get_tweet_embedding(tweet, embedding_model,dimension):
    tweet_embedding = np.zeros(dimension)
    processed_tweet = process_tweet_stem(tweet)
    print(processed_tweet)
    for word in processed_tweet:
        if embedding_model.get_index(word,False):
            tweet_embedding += embedding_model.get_vector(word,norm=0)
    return tweet_embedding


In [ ]:
tweets_embedded = [get_tweet_embedding(tweet,glove_vectors,200) for tweet in all_tweets]

Resta, então, separar entre conjunto de treino e teste, será feito teste com estes dados sem pré processamento, escalados com StandardScaler() e normalizados com normalize().

In [ ]:
#Transforma para numpy array
tweets_embedded_np = np.array(tweets_embedded)
#Dados sem nenhuma normalização ou pré processamento desse tipo
tweets_train_set,tweets_test_set = split_dataset(tweets_embedded_np,0.9,0.1)

In [ ]:
#Dados normalizados com StandardScaler()
tweets_embedded_np_scaled = StandardScaler().fit_transform(tweets_embedded_np)
tweets_train_set_scaled,tweets_test_set_scaled = split_dataset(tweets_embedded_np_scaled,0.9,0.1)

In [ ]:
#Dados normalizados com normalize
tweets_embedded_np_normalized = normalize(tweets_embedded_np, axis = 0)
tweets_train_set_normalized,tweets_test_set_normalized = split_dataset(tweets_embedded_np_normalized,0.9,0.1)

### k-means

Classes e funções implementadas para aplicar o k-means:

In [ ]:
class Point:
    def __init__(self, dimensions, point_index, cluster_index):
        self.dimensions = dimensions
        self.point_index = point_index
        self.cluster_index = cluster_index
        self.distance_to_center = -1

class Cluster:
    def __init__(self, dimensions):
        self.dimensions = dimensions
        self.points = []

def k_means(data, k):
    clusters = forgy_cluster_initialization(k, data)
    redefined = 1
    interac = 0
    while redefined == 1:
        if (interac%500 == 0):
            print("interaction number ", interac)
        for point in data:
            for i in range(len(clusters)):
                if (point.cluster_index == -1):
                    point.cluster_index = i
                cluster_from_current_point = clusters[point.cluster_index]
                if (distance.euclidean(point.dimensions, clusters[i].dimensions) < distance.euclidean(point.dimensions, cluster_from_current_point.dimensions)):
                    point.cluster_index = i
                    point.distance_to_center = distance.euclidean(point.dimensions, clusters[i].dimensions)
            clusters[point.cluster_index].points.append(point)
        redefined = redefine_clusters_center(clusters)
        interac += 1
    return clusters
    
def random_initialization(k, data):
    data_points = []
    for point in data:
        data_points.append(point.dimensions)

    max_value = np.max(np.array(data_points)[:,0])
    d = len(point.dimensions)
    clusters = []
    for _ in range(k):
        clusters.append(Cluster([random.uniform(0, max_value)] * d))
    return clusters

def forgy_cluster_initialization(k, data):
    clusters = []
    for _ in range(k):
        random_index = random.randint(0, len(data))
        clusters.append(Cluster(data[random_index].dimensions))
    return clusters

def redefine_clusters_center(clusters):
    redefined = 0
    for cluster in clusters:
        if (len(cluster.points) == 0):
            continue

        d = len(cluster.dimensions)
        sums = [0] * d

        for point in cluster.points:
            for i in range(d):
                sums[i] += point.dimensions[i]

        means = [0] * d
        n = len(cluster.points)
        for i in range(d):
            means[i] = round(sums[i]/n, 300)
            if (means[i] - cluster.dimensions[i] > 0.000001):
                cluster.dimensions[i] = means[i]
                redefined = 1
        if (redefined == 1):
            cluster.points.clear()
    return redefined

def plot_2d_data(clusters):
    colors = ["green","blue","yellow","pink","orange","purple","beige","brown","gray","cyan","magenta", "black"]
    for cluster in clusters:
        plt.plot(cluster.dimensions[0], cluster.dimensions[1], color = 'red', marker = 'o')
        for point in cluster.points:
            plt.scatter(point.dimensions[0], point.dimensions[1], color = colors[point.cluster_index], alpha = 0.5)
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.show()

def elbow_method(min_k, max_k, data):
    costs = []
    for i in range(min_k, (max_k + 1)):
        clusters = k_means(data, i)
        costs.append(cost_evaluation(clusters))
    plt.plot(range(1, max_k + 1), costs, color = 'blue', marker = 'o', alpha = 0.5)
    plt.xlabel('No. clusters')
    plt.ylabel('Cost')
    plt.show()
    return

def silhouette_analysis():
    return

def cost_evaluation(clusters):
    j = 0
    for cluster in clusters:
        for point in cluster.points:
            j += pow(point.distance_to_center, 2)
    return j
def print_tweets_clusters(clusters, all_tweets_table):
    for i in range(len(clusters)):
        print('Cluster número ', i)
        for j in range(10):
            #Imprime 10 tweets aleatorios que pertencem ao cluster i
            print(all_tweets_table[clusters[i].points[random.randrange(0,len(clusters[i].points))].point_index])
        print('--------próximo cluster----------')
    


**Tarefa 2D padrão:**

Será feito o teste com os dados normalizados com a função normalize do sklearn, StandardScaler e sem realizar nenhum ajuste. Além disso, como métrica de avaliação será plotado uma análise dos coeficientes de Silhouette, o valor médio do mesmo e também a função custo, que é dada pela soma da distância ao quadrado de cada ponto ao centro do seu respectivo cluster. Desta forma é possível avaliar os clusters internamente (densidade) e externamente (separação), pois ambos são englobados pelo coeficiente de silhouette, além da função custo, que é uma outra forma de avaliar a densidade dos clusters. Os outros testes feitos ao longo do trabalho também seguirão esse padrão. Apesar da função custo não representar com a mesma precisão a densidade dos clusters quando o método utilizado faz o agrupamento em outros formatos além do esférico, essa métrica ainda foi mantida para fins de comparação.

In [ ]:
#Aplicar nos sets de treino
#Dados sem pré processamento:
tic = perf_counter()
points = []
for i in range(len(train_set)):
    points.append(Point(train_set[i], i, -1))
numero_de_clusters = 3
clusters = k_means(points, numero_de_clusters)
toc = perf_counter()
time_2D_sp = (toc-tic)*1000
print (f"It took {time_2D_sp:.2f}ms to run the algorithm.")
cost = cost_evaluation(clusters)
print(cost)
plot_2d_data(clusters)
avg_sa_2D_sp = plot_silhouette_analysis(clusters)

In [ ]:
#Dados normalizados com StandardScaler
tic = perf_counter()
points = []
for i in range(len(train_set_scaled)):
    points.append(Point(train_set_scaled[i],i, -1))
numero_de_clusters = 3
clusters = k_means(points, numero_de_clusters)
toc = perf_counter()
time_2D_standard = (toc-tic)*1000
print (f"It took {time_2D_standard:.2f}ms to run the algorithm.")
cost = cost_evaluation(clusters)
print(cost)
plot_2d_data(clusters)
avg_sa_2D_standard = plot_silhouette_analysis(clusters)

In [ ]:
#Dados normalizador com normalize
tic = perf_counter()
points = []
for i in range(len(train_set_normalized)):
    points.append(Point(train_set_normalized[i],i, -1))
numero_de_clusters = 3
clusters = k_means(points, numero_de_clusters)
toc = perf_counter()
time_2D_normalized = (toc-tic)*1000
print (f"It took {time_2D_normalized:.2f}ms to run the algorithm.")
cost = cost_evaluation(clusters)
print(cost)
plot_2d_data(clusters)
avg_sa_2D_normalized = plot_silhouette_analysis(clusters)

In [ ]:
#Dados

Percebe-se então que os dois métodos de normalização fizeram com que o resultado fosse o esperado, enquanto que a técnica aplicada nos dados sem processamento gerou clusters longe do esperado e um custo muito alto. Além disso, percebe-se também que o custo utilizando a função normalize foi menor. Esse resultado faz sentido pois a normalização apresenta melhores resultados quando comparada ao StandardScaler quando os dados do problema não seguem uma distruibuição Gaussiana.

Nos testes feitos anteriormente assumiu-se k = 3 por saber-se previamente o formato dos dados, mas esse não será o caso no próximo conjunto de dados, portanto foi implementado o método do elbow para estimativa do k. É necessário, então, testá-lo no conjunto de dados em 2D antes de aplicá-lo nos dados mais numerosos.

In [ ]:
points = []
for i in range(len(train_set_normalized)):
    points.append(Point(train_set_normalized[i],i, -1))
elbow_method(1,10,points)

Percebe-se, então, que k = 3 é o melhor valor para k de acordo com o método do elbow e, portanto, o método aparenta ser eficaz e pode ser utilizado no segundo conjunto de dados.

**2º conjunto de dados: tweets.**

Será feito o teste com os dados normalizados com a função normalize do sklearn, StandardScaler e sem realizar nenhum ajuste. Primeiramente será rodado o algoritmo com k = 2 a fim de encontrar a melhor forma de pré processar os dados, já que sabe-se a princípio que há tweets positivos e negativos no conjunto e esse pode ser um bom número de clusters. Em seguida, será rodado o método do elbow para verificar se essa hipótese é verdadeira. Sabe-se, também, que a clusterização nestes dados é extremamente dependente da forma como embedding foi feito e, por conta disso, poderá ser testado um outro embedding.

Os clusters resultantes serão analisados ao gerar word clouds, onde as palavras mais frequentes de cada clusters serão impressas em um tamanho maior, possibilitando, então, identificar possíveis tópicos predominantes em cada cluster.

In [ ]:
def print_word_cloud(clusters,all_tweets_table):
    tweets_processed = []
    for i in range(len(all_tweets_table)):
        tweets_processed.append(process_tweet_stem(all_tweets_table[i]))
    for i in range(len(clusters)):
        unique_strings = ''
        print('Cluster número ', i, ' tamanho:', len(clusters[i].points))
        for j in range(len(clusters[i].points)):
            unique_strings += ' '+(' ').join(tweets_processed[clusters[i].points[j].point_index])
        wordcloud = WordCloud(width = 1000, height = 500).generate(unique_strings)
        plt.figure(figsize=(15,8))
        plt.imshow(wordcloud)
        plt.axis("off")
        plt.show()
        plt.close()
        print('--------próximo cluster----------')

In [ ]:
#Dados sem pré processamento:
tic = perf_counter()
points = []
for i in range(len(tweets_train_set)):
    points.append(Point(tweets_train_set_scaled[i],i, -1))
numero_de_clusters = 2
clusters = k_means(points, numero_de_clusters)
toc = perf_counter()
time_tweets_sp = (toc-tic)*1000
print (f"It took {time_tweets_sp:.2f}ms to run the algorithm.")
cost = cost_evaluation(clusters)
print(cost)
print_word_cloud(clusters,all_tweets)
avg_sa_tweets_sp = plot_silhouette_analysis(clusters)

In [ ]:
#Usando StandardScaler:
tic = perf_counter()
points = []
for i in range(len(tweets_train_set_scaled)):
    points.append(Point(tweets_train_set_scaled[i],i, -1))
numero_de_clusters = 2
clusters = k_means(points, numero_de_clusters)
toc = perf_counter()
time_tweets_standard = (toc-tic)*1000
print (f"It took {time_tweets_standard:.2f}ms to run the algorithm.")
cost = cost_evaluation(clusters)
print(cost)
print_word_cloud(clusters,all_tweets)
avg_sa_tweets_standard = plot_silhouette_analysis(clusters)

In [ ]:
#Usando normalize:
tic = perf_counter()
points = []
for i in range(len(tweets_train_set_normalized)):
    points.append(Point(tweets_train_set_normalized[i],i, -1))
numero_de_clusters = 2
clusters = k_means(points, numero_de_clusters)
toc = perf_counter()
time_tweets_normalized = (toc-tic)*1000
print (f"It took {time_tweets_normalized:.2f}ms to run the algorithm.")
cost = cost_evaluation(clusters)
print(cost)
print_word_cloud(clusters,all_tweets)
avg_sa_tweets_normalized = plot_silhouette_analysis(clusters)

Por conta da representação dos tweets ser um dos possíveis motivos da clusterização não ter tido um resultado muito bom, será testada a representação Doc2Vec, que considera o tweet como um todo na representação e não faz somente uma média da representação de cada palavra do mesmo. Além disso, esta representação será treinada dentro do nosso próprio conjunto de dados, onde espera-se um resultado mais satisfatório, porém um número maior de tweets seria necessário para uma representação mais precisa.

In [ ]:
documents = [TaggedDocument(process_tweet_stem(doc), [i]) for i, doc in enumerate(all_tweets)]
model = Doc2Vec(documents, vector_size=100, window=2, min_count=1, workers=4)

In [ ]:
document_model_data = [model.dv[i] for i in range(len(all_tweets))]
document_model_data_normalized = normalize(np.array(document_model_data),axis=0)
#Separando entre conjunto de teste e treino
document_model_train_data_normalized, document_model_test_data_normalized = split_dataset(document_model_data_normalized,0.9,0.1)

Primeiro será rodado o método do elbow para encontrar o melhor valor para k.

In [ ]:
points = []
for i in range(len(document_model_train_data_normalized)):
    points.append(Point(document_model_train_data_normalized[i],i, -1))
elbow_method(1,10,points)

Percebe-se então que k=3 é o valor mais indicado.

In [ ]:
#Usando normalize:
tic = perf_counter()
points = []
for i in range(len(document_model_train_data_normalized)):
    points.append(Point(document_model_train_data_normalized[i],i, -1))
numero_de_clusters = 3
clusters = k_means(points, numero_de_clusters)
toc = perf_counter()
time_tweets_document = (toc-tic)*1000
print (f"It took {time_tweets_document:.2f}ms to run the algorithm.")
cost = cost_evaluation(clusters)
print(cost)
for i,cluster in enumerate(clusters):
    print('Tamanho do cluster ', i, 'é: ',len(cluster.points))
print_word_cloud(clusters,all_tweets)
avg_sa_tweets_document = plot_silhouette_analysis(clusters)

### Outro método: DBSCAN


O método DBSCAN foi escolhido para ser implementado, já que o mesmo lida bem com outliers e o dataset dos tweets possui alguns, o que pode ter influenciado nos resultados.

Assim como no k-means, foram definidas as classes DBSCANpoint e DBSCANCluster para facilitarem na implementação do algoritmo. A ckasse DBSCANpoint, utilizada para representar um dado, contém as dimensões do ponto (x,y,z,...), o identificador de cada ponto (utilizado para recuperar os tweets), o cluster a qual ele pertence, sua categoria no DBSCAN (core, outlier ou border), seus vizinhos, o número total de vizinhos, informação sobre a presença de um core point na vizinhança e, por fim, o coeficiente de silhouette associado.

A classe DBSCANCluster, por sua vez, possui somente uma lista 

In [88]:
class DBSCANpoint:
    def __init__(self,dimensions,index):
        self.dimensions = dimensions
        self.point_index = index
        self.cluster = None
        self.category = 'NONE'
        self.neighbours = []
        self.n_neighbours = 0
        self.there_is_core_in_neighbourhood = False
        self.silhouette_coefficient = 0

    def add_neighbour(self,DBSCANpoint):
        self.neighbours.append(DBSCANpoint)
        self.n_neighbours += 1

class DBSCANCluster:
    def __init__(self):
        self.points = []
    def mean_dimensions(self):
        self.dimensions = np.zeros(len(self.points[0].dimensions))
        for point in self.points:
            self.dimensions = np.add(self.dimensions,point.dimensions)
        self.dimensions = self.dimensions/len(self.points)

def dbscan(data,epsilon,M):
    #data is a np.array
    all_points,core_points,outliers = define_type(data,epsilon,M)
    clusters = []
    #form clusters
    for core_point in core_points:
        #Connect all core_points
        for neighbour in core_point.neighbours:
            if neighbour.category == 'CORE':
                if neighbour.cluster == None:
                    #neighbour core point still dont have a cluster_index
                    if (core_point.cluster == None):
                        #both of them dont have a cluster index, so assign them a new one
                        core_point.cluster = DBSCANCluster()
                        neighbour.cluster = core_point.cluster
                        clusters.append(core_point.cluster)
                        core_point.cluster.points.append(core_point)
                        core_point.cluster.points.append(neighbour)
                    else:
                        #core_point already has a cluster    
                        neighbour.cluster = core_point.cluster
                        core_point.cluster.points.append(neighbour)
                else:
                    #neighbour already has a cluster
                    if (core_point.cluster == None):
                        #neighbour has a cluster but the core_point doesn't
                        core_point.cluster = neighbour.cluster
                        neighbour.cluster.points.append(core_point)
                    else:
                        #both of them have a cluster
                        if (neighbour.cluster != core_point.cluster):
                            merge_clusters(clusters,core_point.cluster,neighbour.cluster)
                            

    for core_point in core_points:
        #Assign clusters to the border points
        for neighbour in core_point.neighbours:
            if neighbour.category != 'CORE':
                neighbour.cluster = core_point.cluster
                core_point.cluster.points.append(neighbour)
            
    for cluster in clusters:
        cluster.mean_dimensions()
    return clusters,core_points

def define_type(data,epsilon,M):
    points = []
    core_points = []
    outliers = []
    for i in range(0,len(data)):
        points.append(DBSCANpoint(data[i],i))
    
    for point in points:
        search_neighbours(point,points,epsilon)
        if point.n_neighbours >= M:
            point.category = 'CORE'
            core_points.append(point)
        elif point.there_is_core_in_neighbourhood:
            point.category = 'BORDER'
        else:
            point.category = 'OUTLIER'
            outliers.append(point)
    
    return points,core_points,outliers

def search_neighbours(DBSCANpoint, all_points,epsilon):
    for point in all_points:
        if (distance.euclidean(DBSCANpoint.dimensions,point.dimensions) <= epsilon):
            DBSCANpoint.add_neighbour(point)
            if point.category == 'CORE':
                DBSCANpoint.there_is_core_in_neighbourhood = True

def merge_clusters(clusters_list,cluster_1,cluster_2):
    #all points of cluster 2 are gonna become part of cluster 1
    for point in cluster_2.points:
        point.cluster = cluster_1
        cluster_1.points.append(point)

    clusters_list.remove(cluster_2)

def assign_new_point(DBSCANpoint,core_points, epsilon):
    shortest_dist = 9999
    for core_point in core_points:
        tmp_dist = distance.euclidean(DBSCANpoint.dimensions,core_point.dimensions)
        if (tmp_dist <= epsilon):
            if (tmp_dist < shortest_dist):
                closest_core = core_point
    
    core_point.cluster.append(DBSCANpoint)
    DBSCANpoint.cluster = core_point.cluster

def test_new_data(new_data,core_points,epsilon):
    for point in new_data:
        assign_new_point(point,core_points,epsilon)

   

def dbscan_plot_2d_data(clusters):
    colors = ["green","blue","yellow","pink","orange","purple","beige","brown","gray","cyan","magenta", "black"]
    for cluster in clusters:
        plt.plot(cluster.dimensions[0], cluster.dimensions[1], color = 'red', marker = 'o')
        for point in cluster.points:
            plt.scatter(point.dimensions[0], point.dimensions[1], color = colors[clusters.index(point.cluster)], alpha = 0.5)
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.show()

def plot_different_epsilons(min_epsilon, max_epsilon, step, data, M):
    silhouette_list = []
    epsilon_list = []
    epsilon_tmp = min_epsilon
    while epsilon_tmp <= max_epsilon:
        print('-----------------------------------------------------------------')
        print('For epsilon = ',epsilon_tmp)
        tic = perf_counter()
        clusters_tmp,_ = dbscan(data, epsilon_tmp, M)
        toc = perf_counter()
        total_time = (toc-tic)*1000
        print (f"It took {total_time:.2f}ms to run the algorithm.")
        print('Total: ',len(clusters_tmp),' clusters')
        for i,cluster in enumerate(clusters_tmp):
            print('Tamanho do cluster ', i+1, 'é: ',len(cluster.points))
        clusters_tmp = silhouette_analysis(clusters_tmp)
        silhouette_list.append(plot_silhouette_analysis(clusters_tmp))
        epsilon_list.append(epsilon_tmp)
        epsilon_tmp += step
    plt.plot(epsilon_list, silhouette_list, color = 'blue', marker = 'o', alpha = 0.5)
    plt.xlabel('Epsilon Value')
    plt.ylabel('Average Silhouette value')
    plt.show()
    return

Por conta do DBSCAN depende de dois hiperparâmetros, não é possível aplicar o método elbow como no k-means para encontrar bons valores de M e epsilon. Todavia, uma boa estimativa para o valor de M é a quantidade de dimensoes dos dados, porém foram feitas várias tentativas com o valor de M calculado dessa maneira, mas sem sucesso pois os dados acabavam todos dentro do mesmo cluster, sendo assim M = 2.(dimensão dos dados) foi escolhido. Sendo assim, é possível plotar gráficos para diferentes valores de epsilon e ver a qualidade dos clusters resultantes, a métrica utilizada para medir essa qualidade foi o Coeficiente de Silhouette, já que esse coeficiente engloba coesão e separação.


In [ ]:
tic = perf_counter()
M = 2*train_set_normalized.shape[1]
print("M = ",M)
epsilon = 0.005
clusters_dbscan_2d,_ = dbscan(train_set_normalized, epsilon, M)
toc = perf_counter()
time_dbscan_2d = (toc-tic)*1000
print (f"It took {time_dbscan_2d:.2f}ms to run the algorithm.")
for i,cluster in enumerate(clusters_dbscan_2d):
    print('Tamanho do cluster ', i, 'é: ',len(cluster.points))
dbscan_plot_2d_data(clusters_dbscan_2d)
clusters_dbscan_2d = silhouette_analysis(clusters_dbscan_2d)
plot_different_epsilons(0.002,0.01,0.001,train_set_normalized,M)

In [ ]:
M = 2*document_model_train_data_normalized.shape[1]
print("M = ",M)
plot_different_epsilons(0.001,0.001,0.01,document_model_train_data_normalized,M)

Percebe-se então que o algoritmo não conseguiu encontrar clusters para nenhum dos valores de epsilon tentados. Isso pode ter acontecido por conta dos dados dos tweets estarem todos aglomerados próximos uns dos outros, fazendo com que algoritmos como o k-means e o DBSCAN não consiga separá-los em clusters menores, nesse caso, técnicas diferentes podem ter um melhor resultado, como as baseadas em probabilidade ou hierarquia. Isso mostra como uma heurpistica não é garantia de nada, e sim somente um guia.

Além disso, o tempo de execução do DBSCAN foi extremamente alto para os tweets, especialmente por conta do alto número de dimensões dos dados, fazendo com que não fosse possível realizar testes para um grande número de combinações de M e epsilon, os testes realizados já exigiram mais de 15 horas de processamento. Sendo assim, uma melhora neste quesito é esperada ao aplicar uma redução de dimensionalidade nos dados.

Também, essa "aglomeração" dos tweets pode ter sido causada por uma má representação vetorial dos mesmos. Apesar de terem sido experimentados diferentes modelos de transformação, o ideal seria treinar um modelo próprio em um corpus maior, 10000 tweets não é um número muito grande de dados, principalmente por serem textos curtos, onde muitos caracteres ainda são removidos na etapa de pré processamento. Um exemplo de experimento que poderia ser feito e que provavelmente teria melhores resultados é treinar um modelo de representação baseado em atenção, como BERT, onde algumas etapas do pré processamento poderiam até ser desconsideradas, já que esse tipo de modelo pode captar o significado de termos antes considerados como irrelevantes, como algumas stopwords.

## Parte 2 - Redução de dimensionalidade

A biblioteca scikit lean foi utilizada para aplicar PCA aos dados. Vamos primeiro observar os dados graficamente ao reduzi-los para 2 e 3 dimensões.

In [ ]:
from sklearn.decomposition import PCA
def red_dim(data,n_comp):
    pca = PCA(n_components=n_comp)
    pca.fit(data)
    new_data = pca.transform(data)
    print("Dimensionalidade reduzida, proporção de variância em cada componente é:")
    print(pca.explained_variance_ratio_)
    return new_data

#
tweets_new_2D = red_dim(train_set_normalized,2)
scatter_data_2D(tweets_new_2D)
tweets_new_3D = red_dim(train_set_normalized,3)
scatter_data_3D(tweets_new_3D)

Percebe-se, então, que por volta de 58% da informação dos dados pode ser representada utilizando somente 1 feature, sendo assim, o resultado das técnicas de clusterização não devem mudar muito entre dados após redução para 1 dimensão, 2 dimensões e 3 dimensões. Portanto, será aplicado o k-means nos dados reduzidos para 1,2 e 3 dimensões, onde primeiramente será aplicado o método do elbow para encontrar um bom valor para k.

Ao observar a representação gráficas destes dados após redução para 2D ou 3D, é possível perceber a presença de um cluster pequeno, que não foi identificado tanto pelo k-means quanto pelo dbscan

In [ ]:
tweets_new_1D = red_dim(document_model_data_normalized,1)
tweets_new_1D_train,tweets_new_1D_test = split_dataset(tweets_new_1D,0.9,0.1)
tweets_new_2D = red_dim(document_model_data_normalized,2)
tweets_new_2D_train,tweets_new_2D_test = split_dataset(tweets_new_2D,0.9,0.1)
tweets_new_3D = red_dim(document_model_data_normalized,3)
tweets_new_3D_train,tweets_new_3D_test = split_dataset(tweets_new_3D,0.9,0.1)


In [ ]:
points = []
for i in range(len(tweets_new_1D_train)):
    points.append(Point(tweets_new_1D_train[i],i, -1))

elbow_method(1,6,points)

In [ ]:
points = []
for i in range(len(tweets_new_1D_train)):
    points.append(Point(tweets_new_1D_train[i],i, -1))
numero_de_clusters = 4
tic = perf_counter()
clusters = k_means(points, numero_de_clusters)
toc = perf_counter()
time_tweets_1d_kmeans = (toc-tic)*1000
print (f"It took {time_tweets_1d_kmeans:.2f}ms to run the algorithm.")
cost = cost_evaluation(clusters)
print(cost)
print_word_cloud(clusters,all_tweets)
clusters = silhouette_analysis(clusters)
avg_sa_tweets_1d = plot_silhouette_analysis(clusters)

In [ ]:
#Dados normalizador com normalize
points = []
for i in range(len(tweets_new_2D_train)):
    points.append(Point(tweets_new_2D_train[i],i, -1))

elbow_method(1,6,points)

In [ ]:
points = []
for i in range(len(tweets_new_2D_train)):
    points.append(Point(tweets_new_2D_train[i],i, -1))
numero_de_clusters = 3
tic = perf_counter()
clusters = k_means(points, numero_de_clusters)
toc = perf_counter()
time_tweets_2d_kmeans = (toc-tic)*1000
print (f"It took {time_tweets_2d_kmeans:.2f}ms to run the algorithm.")
cost = cost_evaluation(clusters)
print(cost)
clusters = silhouette_analysis(clusters)
avg_sa_tweets_2d = plot_silhouette_analysis(clusters)
plot_2d_data(clusters)

In [ ]:
points = []
for i in range(len(tweets_new_3D_train)):
    points.append(Point(tweets_new_3D_train[i],i, -1))

elbow_method(1,6,points)

In [ ]:
points = []
for i in range(len(tweets_new_3D_train)):
    points.append(Point(tweets_new_3D_train[i],i, -1))
numero_de_clusters = 2
tic = perf_counter()
clusters = k_means(points, numero_de_clusters)
toc = perf_counter()
time_tweets_3d_kmeans = (toc-tic)*1000
print (f"It took {time_tweets_3d_kmeans:.2f}ms to run the algorithm.")
cost = cost_evaluation(clusters)
print(cost)
clusters = silhouette_analysis(clusters)
avg_sa_tweets_3d = plot_silhouette_analysis(clusters)

Testando agora o dbscan, como o número de dimensões neste caso é bem pequeno, então M será estimado como um valor maior para evitar a inclusão de outliers nos clusters.

Dados em 1D

In [ ]:
tic = perf_counter()
M = 50
print("M = ",M)
epsilon = 0.005
clusters_dbscan_tweets_1D,_ = dbscan(tweets_new_1D_train, epsilon, M)
toc = perf_counter()
time_dbscan_tweets_1d = (toc-tic)*1000
print (f"It took {time_dbscan_tweets_1d:.2f}ms to run the algorithm.")
print_word_cloud(clusters_dbscan_tweets_1D,all_tweets)

In [ ]:
print_word_cloud(clusters_dbscan_tweets_1D,all_tweets)

Dados em 2D

In [ ]:
tic = perf_counter()
M = 2*tweets_new_2D_train.shape[1]
print("M = ",M)
epsilon = 0.005
clusters_dbscan_tweets_2D,_ = dbscan(tweets_new_2D_train, epsilon, M)
toc = perf_counter()
time_dbscan_tweets_2d = (toc-tic)*1000
print (f"It took {time_dbscan_tweets_2d:.2f}ms to run the algorithm.")
print_word_cloud(clusters_dbscan_tweets_2D,all_tweets)

Dados em 3D

In [ ]:
tic = perf_counter()
M = 2*tweets_new_3D_train.shape[1]
print("M = ",M)
epsilon = 0.005
clusters_dbscan_tweets_3D,_ = dbscan(tweets_new_3D_train, epsilon, M)
toc = perf_counter()
time_dbscan_tweets_3d = (toc-tic)*1000
print (f"It took {time_dbscan_tweets_3d:.2f}ms to run the algorithm.")
print_word_cloud(clusters_dbscan_tweets_3D,all_tweets)